In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: parthtiwari27
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


100%|██████████| 25.7M/25.7M [00:00<00:00, 154MB/s]

In [4]:
import numpy as np
import pandas as pd


In [5]:
df = pd.read_csv('/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df.shape

(50000, 2)

In [8]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [9]:
df.replace({'sentiment':{'positive':1,'negative':0}},inplace = True)

<ipython-input-9-63b0212fca70>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'sentiment':{'positive':1,'negative':0}},inplace = True)


In [10]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
train_data, test_data = train_test_split(df, test_size = 0.25, random_state=42)

In [14]:
train_data.shape

(37500, 2)

In [15]:
test_data.shape

(12500, 2)

In [16]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [17]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [18]:
X_train

array([[   0,    0,    0, ...,    1,   13,  190],
       [ 148,   48,    1, ...,  103,    3,  577],
       [   0,    0,    0, ...,  292,   29, 2104],
       ...,
       [   0,    0,    0, ..., 1614,    2,  593],
       [   0,    0,    0, ...,  246,  103,  125],
       [   0,    0,    0, ...,   70,   72, 2069]], dtype=int32)

In [19]:
X_test

array([[   0,    0,    0, ...,  989,  719,  155],
       [  12,  162,   59, ...,  377,    7,    7],
       [   0,    0,    0, ...,   51, 1095,   97],
       ...,
       [   0,    0,    0, ...,  115,    9,   86],
       [   0,    0,    0, ...,    4,  280, 2160],
       [   0,    0,    0, ...,    4,   29,   55]], dtype=int32)

In [20]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [21]:
Y_train

,sentiment
27434,0
13400,0
883,0
7303,0
45124,1
...,...
11284,1
44732,1
38158,0
860,1


In [22]:
Y_test

,sentiment
33553,1
9427,1
199,0
12447,1
39489,0
...,...
9771,0
34737,0
30390,1
6797,1


In [23]:
model = Sequential()

model.add(Embedding(input_dim=5000, output_dim=128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(None, 200))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [25]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.25)

Epoch 1/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 177s 395ms/step - accuracy: 0.7304 - loss: 0.5346 - val_accuracy: 0.8534 - val_loss: 0.3537
Epoch 2/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 182s 414ms/step - accuracy: 0.8532 - loss: 0.3539 - val_accuracy: 0.8322 - val_loss: 0.4039
Epoch 3/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 187s 425ms/step - accuracy: 0.8728 - loss: 0.3091 - val_accuracy: 0.8570 - val_loss: 0.3287
Epoch 4/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 210s 443ms/step - accuracy: 0.8976 - loss: 0.2587 - val_accuracy: 0.8731 - val_loss: 0.3185
Epoch 5/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 192s 436ms/step - accuracy: 0.9133 - loss: 0.2231 - val_accuracy: 0.8717 - val_loss: 0.3647


In [26]:
model.save("model.h5")

In [27]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [28]:
loss, accuracy = model.evaluate(X_test, Y_test)

391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.8808 - loss: 0.3400


In [29]:
print(loss)

0.3371778428554535


In [30]:
print(accuracy)

0.8831999897956848


<h3>BUILDING A PREDICTION FUNCTION </h3>
When this function is given a text as input it will predict whether the sentiment is good or bad

In [31]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [32]:
predictive_system("A good movie overall")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step


'positive'

In [33]:
predictive_system("Boring and slow paced")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


'negative'

In [36]:
predictive_system("Stunning visuals and brilliant acting and direction however the music was not upto the mark")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


'positive'